In [3]:
import torch
import torch.nn as nn

In [13]:
class Attention(nn.Module):
    def __init__(self, embed_size, heads):
        super(Attention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert (
            self.head_dim * heads == embed_size
        ), "Embed size needs to be divisible by heads"

        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)

    def forward(self, values, keys, query, mask=None):
        N = query.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        # Split the embedding into self
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        query = query.reshape(N, query_len, self.heads, self.head_dim)

        energy = torch.einsum("nqhd,nkhd->nhqk", [query, keys])
        # queries shape: (N, query_len, heads, head_dim)
        # keys shape: (N, key_len, heads, head_dim)
        # energy shape: (N, heads, query_len, key_len)


        # For demonstration purposes, we'll just return the reshaped tensors
        return values, keys, query

In [15]:
# Example usage
# if __name__ == "__main__":
embed_size = 8
heads = 2
batch_size = 1
seq_length = 4

# Create dummy data
values = torch.randn(batch_size, seq_length, embed_size)
keys = torch.randn(batch_size, seq_length, embed_size)
query = torch.randn(batch_size, seq_length, embed_size)

print("Original shapes:")
print("Values shape:", values.shape)
print("Keys shape:", keys.shape)
print("Query shape:", query.shape)

# Create the Attention layer
attention = Attention(embed_size, heads)

# Forward pass
reshaped_values, reshaped_keys, reshaped_query = attention(values, keys, query)

print("\nReshaped shapes:")
print("Reshaped Values shape:", reshaped_values.shape)
print("Reshaped Keys shape:", reshaped_keys.shape)
print("Reshaped Query shape:", reshaped_query.shape)

Original shapes:
Values shape: torch.Size([1, 4, 8])
Keys shape: torch.Size([1, 4, 8])
Query shape: torch.Size([1, 4, 8])

Reshaped shapes:
Reshaped Values shape: torch.Size([1, 4, 2, 4])
Reshaped Keys shape: torch.Size([1, 4, 2, 4])
Reshaped Query shape: torch.Size([1, 4, 2, 4])
